# POC Power BI TS Formation

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl 

## 1) Déclaration des fonctions et classes exploitées dans le code

In [52]:
class Datamanagement:
    def __init__(self):
        self.source = {} # Contient le nom et la localisation des fichiers sources
        self.dim = {} # Contient le nom et les dataframes des tables de dimension
        self.fact = {} # Contient le nom et les dataframes liés aux tables de faits
        self.changed_keys = {}
    
    def import_csv(self, filename, engine_val=None, encoding_val='utf-8', sep_val=';',low_memory_val=False):
        self.source[filename[:-4]] = pd.read_csv(r'Data/'+ filename, engine=engine_val, encoding=encoding_val, sep=sep_val, low_memory=low_memory_val)
        return self.source[filename[:-4]]
    
    def update_key(self, changed_keys, filename):
        self.changed_keys[filename] = changed_keys
        self.source[filename].rename(columns=changed_keys , inplace=True)
        return self.source[filename]
    
    def import_dim(self,dimname, dimdataframe):
        self.dim[dimname]=dimdataframe
        
    def import_fact(self,factname, factdataframe):
        self.fact[factname] = factdataframe
    
    def export(self):
        for key, value in self.dim.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-8')
            
        for key, value in self.fact.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-16', index=False)
    

### Génération/Mise à jour du fichier setting

In [53]:
def import_data():
    all_dir = glob.glob("Data/*.csv")    
    all_files_name = [x[5:] for x in all_dir]
    all_files_name = [x[:-4] for x in all_files_name]
    df_files_col = pd.DataFrame()
    p = 1
    
    for dir in all_dir:
        df = pd.read_csv(dir, encoding='utf-8', sep=';')
        
        for i, col in enumerate(df.columns):
            df_files_col.at[p, 'Nom fichier source'] = dir[5:-4]
            df_files_col.at[p,'Nom champ source'] = col
            df_files_col.at[p,'Type'] = str(df[col].dtypes)
            df_files_col.at[p,'Synthèse'] = str(df[col].describe())
            df_files_col.at[p, 'Répertoire'] = dir
            p = p + i
    
    df_dir_files = df_files_col.copy()
    df_dir_files = df_dir_files[["Nom fichier source", "Répertoire"]].drop_duplicates()
    
    if path.exists("Settings & documentation\Settings.xlsx"):
        pass

    else:

        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='xlsxwriter')
        workbook  = writer.book
        df_dir_files.to_excel(writer, sheet_name='Fichiers Source', index=False)
        
        df_files_col[['Nom fichier source', 'Nom champ source', 'Type', 'Synthèse' ]].to_excel(writer, sheet_name='Fichiers et colonnes source', index=False)
                
        worksheet_FCS = writer.sheets["Fichiers et colonnes source"]
        worksheet_FS = writer.sheets["Fichiers Source"]
        #worksheet1 = workbook.add_worksheet("Mapping données")
        
        cell_format_FCS = workbook.add_format() 
        cell_format_FCS.set_text_wrap()
        cell_format_FCS.set_align('center')
        cell_format_FCS.set_align('vcenter')
        
        cell_format_FS = workbook.add_format()    
        cell_format_FS.set_align('left')
                
        worksheet_FCS.set_column('A:B', 30, cell_format_FCS)
        worksheet_FCS.set_column('C:C', 15, cell_format_FCS)
        worksheet_FCS.set_column('D:D', 25, cell_format_FCS)
        worksheet_FS.set_column('A:A', 40, cell_format_FS)
        worksheet_FS.set_column('B:B', 60, cell_format_FS)

        writer.save()
        workbook.close()

# Fonction créant la feuille Mapping dim dans le fichier Settings
# Cette feuille contient l'ensemble des valeurs de dimension et permet de réaliser un mapping pour changer le nom des attributs

def add_sheet_mapcol(DicDataframe):
    
    df = pd.DataFrame(columns=["Nom dimension", "Nom colonne"],data=[])
    
    for key, value in DicDataframe.items():        
        #if len(key) >= 3:
        #    if key[:3] == "dim":
        for i, col in enumerate(value.columns):
            df.at[i, "Nom dimension"] = key
            df.at[i, "Nom colonne"] = col
            df.at[i, "Nouveau nom"] = ""
            df.at[i, "A mapper"] = "Non"                
      
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    df.to_excel(writer, sheet_name="Mapping dim colonne",engine='openpyxl',index=False)     
    writer.save()
    writer.close()
    
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    workbook1["Mapping dim colonne"].column_dimensions["A"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["B"].width = 20
    workbook1["Mapping dim colonne"].column_dimensions["C"].width = 20
    
    writer.save()
    writer.close()
    
def update_sheet_mapcol(DicDataframe):
    existing_colmap =  pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name='Mapping dim colonne') # Contient les valeurs de mapping dim colonne
    fresh_col = pd.DataFrame(columns=["Nom dimension", "Nom colonne"], data=[]) # Contient l'ensemble des nouvelles valeurs pour Mapping dim colonne
    existing_colmap["Comparaison"] = existing_colmap["Nom dimension"] + existing_colmap["Nom colonne"] # Création de la colonne "Comparaison" servant à comparer les lignes
    to_keep = [] # Contient la liste des valeurs de la colonne "Comparaison" à conserver dans existing_colmap
    to_add = [] # Contient la liste des valeurs de la colonne "Comparaison" à rajouter
    p=0
    
    for key, value in DicDataframe.items():
        
        for i, col in enumerate(value.columns):
            fresh_col.at[p, "Nom dimension"] = key
            fresh_col.at[p, "Nom colonne"] = col
            fresh_col.at[p, "Nouveau nom"] = ""
            fresh_col.at[p, "A mapper"] = "Non"
            p+=1
 
    fresh_col["Comparaison"] = fresh_col["Nom dimension"] + fresh_col["Nom colonne"]
    to_keep = list(fresh_col["Comparaison"])
    existing_colmap = existing_colmap.loc[existing_colmap["Comparaison"].isin(to_keep)]
    existing_colmap.reset_index(drop=True, inplace=True)
  
    # On détermine la liste des nouvelles valeurs absentes de la feuille de mapping
    
    for val in to_keep:
        if val in list(existing_colmap["Comparaison"]):
            pass
            
        else:
            to_add.append(val)
    
    fresh_col = fresh_col.loc[fresh_col["Comparaison"].isin(to_add)]
    fresh_col.reset_index(drop=True, inplace=True)
    
    result = existing_colmap.append(fresh_col)
    
    # On exporte le résultat dans l'onglet Mapping dim colonne de Setting
    result = result.drop(["Comparaison"], axis=1)
    result = result.sort_values(["Nom dimension", "Nom colonne"]) 
    export_to_settings(result, "Mapping dim colonne",2)
    
    return result
   
    
import_data()

my_data = Datamanagement()

In [54]:
# Fonction permettant d'exporter un dataframe vers un onglet de setting avec un formatage automatique des colonnes
def export_to_settings(dataframe, sheetname, position = None):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    numbers = range(0,26,1)
    dic_letters = {} # Dictionnaire contenant les lettres de l'alphabet    
    val_max = [] # Liste contenant la taille des valeurs les plus longues
    
    # Création d'un dictionnaire stockant les lettres de l'alphabet et leur position afin d'alimenter les fonctions openpyxl
    for i in range(len(numbers)): 
        dic_letters[numbers[i]] = letters[i]
    
    # Alimentation de val_max
    for col in dataframe.columns:
        max = (dataframe[col].astype(str)).str.len().max()        
        if max != 0:
            val_max.append(int(max)+1)
        else:
            val_max.append(15)
            
   # Ouverture du fichier setting et création de l'onglet sheetname

    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    list_sheet = workbook1.get_sheet_names() # Récupère la liste des feuilles
  
    if sheetname in list_sheet: # Si l'onglet existe déjà on va le supprimer pour le remplacer
        std = workbook1.get_sheet_by_name(sheetname)
        workbook1.remove_sheet(std)
    
    dataframe.to_excel(writer, sheet_name= sheetname, engine='openpyxl', index=False)     
    writer.save()
    writer.close()
    
    # Gère la position de la feuille
    
    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    
    if position == None:        
        pos = len(list_sheet) - 1
    else:
        pos = position
    
    sheets = workbook1._sheets
        
    sheet = sheets.pop(len(list_sheet) - 1)
    sheets.insert(pos, sheet)
    
    writer.save()
    writer.close()
    
    
   # Réouverture du fichier setting

    workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
    writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
    writer.book = workbook1
    
    # Aliemntation du fichier excel avec les données du dataframe
    
    for i, width in enumerate(val_max):
        if width < 11:
            workbook1[sheetname].column_dimensions[dic_letters[i]].width = 12
        else:
            workbook1[sheetname].column_dimensions[dic_letters[i]].width = width
            
    # Sauvegarde et fermeture du fichier excel
    writer.save()
    writer.close()        

### Fonction mapping des noms de colonnes

In [55]:
# Fonction transformant le nom des colonnes suivant le mapping effectué dans l'onglet Mapping dim colonne du fichier setting
# Entrée: Dictionnaire key: Nom dataframe  Value : Dataframe
# Sortie un dictionnaire "nom"/Dataframe avec les valeurs des colonnes actualisées suivant l'onglet de mapping "Mapping dim colonne" du fichier settings

def map_col(dict_dataframe):
    md = pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name="Mapping dim colonne")
    mdf = md["Nom dimension"].unique()
    dic_map = {}
    for dim in mdf:
        map = md.loc[(md["Nom dimension"] == dim ) & (md["Nouveau nom"].notnull()== True),["Nom colonne","Nouveau nom"]]
        map.reset_index(drop=True, inplace=True)
            
        for i, val in enumerate(map.iterrows()):           
            dict_dataframe[dim].rename(columns={map.at[i, "Nom colonne"]:map.at[i,"Nouveau nom"]},inplace=True)
     
    return dict_dataframe


In [56]:
# Fonction générant une liste des valeurs des champs à mapper. Chaque type de valeur sera extraite.
# Etape 1: Vérification si l'onglet "Mapping données" existe

def mapp_data(dict_dataframe):
    result = pd.DataFrame()
    
    try:       
        to_mapp_data = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données") # Lecture de la feuille Mapping données
        to_mapp_data = to_mapp_data.loc[to_mapp_data["Valeurs cible"] != '']
        to_mapp_data.reset_index(drop=True, inplace=True)
        
        if to_mapp_data.empty == False:
            for i, val in enumerate(to_mapp_data.iterrows()):
                dict_dataframe[to_mapp_data.at[i,"Table"]][to_mapp_data.at[i,"Colonnes"]].replace(to_replace =to_mapp_data.at[i,"Valeurs actuelles"], 
                 value = to_mapp_data.at[i,"Valeurs cible"], inplace=True)
            
    except:
            mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
            mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
            mapp_col["Colonne"] = ""
            mapp_col.reset_index(inplace=True, drop=True)
            mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
            p=1
            if mapp_col.empty == False:
                for i, val in enumerate(mapp_col.iterrows()):
                    if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                        mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                    else:
                         mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]

                for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):
                    
                    if (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                        list_val = (dict_dataframe[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                        nom_col = mapp_col.at[i, "Colonne"]
                        nom_table = mapp_col.at[i, "Nom dimension"]

                    if len(list_val) != 0:
                        for i, val in enumerate(list_val):                           
                            mapp_val.at[p,"Table"] = nom_table
                            mapp_val.at[p,"Colonnes"] = nom_col
                            mapp_val.at[p,"Valeurs actuelles"] = val
                            mapp_val.at[p,"Valeurs cible"] = ""
                            p += 1
                            
            if mapp_val.empty:                
                pass            
            else:
                export_to_settings(mapp_val,"Mapping données")            



## 2) Import des données

In [57]:
%%time
### Ajouter les tables à intégrer ici
tcw = my_data.import_csv('191119_TrainingCollectiveWishes.csv') # Demandes collectives non affectées à un plan de formation
tiw = my_data.import_csv('191119_TrainingIndividualWishes.csv') # Demandes individuelles non affectées à un plan de formation
tp = my_data.import_csv('191119_TrainingPlan.csv') # Plans de formation
tpcw = my_data.import_csv('191119_TrainingPlanCollectiveWishes.csv') # Demande de formation collectives prises en charge dans un plan de formation
tpiw = my_data.import_csv('191119_TrainingPlanIndividualWishes.csv') # Demande de formation individuelles prises en charge dans un plan de formation
tr = my_data.import_csv('191119_TrainingRegister.csv') # Table recensant les inscription aux sessions de formation 
ts = my_data.import_csv('191119_TrainingSessions.csv') # Table recensant les sessions de formation 
tsc = my_data.import_csv('191119_TrainingStageCost.csv') # Table contenant les coûts des stages
tsv2 = my_data.import_csv('191119_TrainingStagev2.csv') # Table contenant les stages
emp = my_data.import_csv('191119_Employees.csv') # Table maître employé
empc = my_data.import_csv('191119_EmployementContract.csv') # Table contrat employé
indO = my_data.import_csv('191119_IndividualOrganization.csv') # Table organisation employé
indpp =my_data.import_csv('191119_IndivPPCsNew.csv') # ?
#setting = pd.read_excel(r'Data/Settings.xlsx', sheets="Mapp_data") # Table de paramétrage servant à mapper les données

Wall time: 663 ms


## 3) Renommage des clés fonctionnelles

In [58]:
%%time
ts = my_data.update_key({'clientcode':'SessionID', 'coursecode':'FormationID', 'startdate':'SessionDate'},'191119_TrainingSessions')
tsv2 = my_data.update_key({'clientcode':'SessionID'},'191119_TrainingStagev2')
tr = my_data.update_key({'clientcode':'SessionID', 'traineusername':'USERNAME'}, '191119_TrainingRegister')
tp = my_data.update_key({'plancode':'PlanID'}, '191119_TrainingPlan')
emp = my_data.update_key({'username':'USERNAME'}, '191119_Employees')
tcw = my_data.update_key({'clientcode':'WishID','coursecode':'FormationID','username':'USERNAME'}, '191119_TrainingCollectiveWishes')
tiw = my_data.update_key({'username':'USERNAME', 'employeenumber':'ZY00.MATCLE', 'clientcode':'WishID', 'coursecode':'FormationID'},'191119_TrainingIndividualWishes')
tpcw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'}, '191119_TrainingPlanCollectiveWishes')
tpiw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'} ,'191119_TrainingPlanIndividualWishes')

Wall time: 29 ms


## 4) Alimentation des dictionnaires contenant les colonnes et dataframe

In [59]:
%%time
my_dict_dataframe = {"tcw": tcw, "tiw": tiw, "tp" : tp, "tpcw" : tpcw, "tr" : tr, "ts" : ts, "tsc" : tsc, "tsv2" : tsv2, "emp" : emp,
           "empc" : empc, "indO": indO, "indpp": indpp}

Wall time: 14 ms


## 5) Creation des tables de dimension

### Création de la table dimension demandes de formations

In [60]:
%%time
# Génération de la table de dimension Souhaits de formation à partir des tables 191119_TrainingCollectiveWishes et 191119_TrainingIndividualWishes 
dim_wish = tcw.append(tiw,sort=True)
dim_wish.reset_index(drop=True, inplace=True)
dim_wish["Wish_key"] = dim_wish.index
dim_wish.loc[dim_wish.nbtrainees.isna(), "nbtrainees"] = 1
dim_wish.index.name="key_wish"
#dim_wish = df_mapp(dim_wish)
my_data.import_dim("Souhaits de formation", dim_wish)

#y_data.dim["dim_wish_table"] = dim_wish
#my_dict_dataframe["dim_wish_table"]= dim_wish
#wish_table.to_excel(r'Transformed data\wish_table.xlsx')

Wall time: 50 ms


### Création de la table dimension plan de formation

In [61]:
%%time
# Génération de la table plan de formation
dim_plan = tp
val_empty = {"PlanID":"0", "nom du plan" : "Pas de plan"} # On insère une ligne pour affecter par la suite les demandes non affectées à une clé plan
dim_plan= dim_plan.append(val_empty,ignore_index=True) 
dim_plan.index.name="key_plan"
my_data.import_dim("Plan de formation", dim_plan)

Wall time: 8 ms


### Traitement du mapping des colonnes et des valeurs

In [62]:
%%time
#Verification si la colonne Mapping dim existe dans le fichier setting, si non on va la créer
try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")   
    Mapp_sheet = True    
except:
    Mapp_sheet = False
    
    
if Mapp_sheet == True:
    update_sheet_mapcol(my_data.dim)
    pass
else:    
    add_sheet_mapcol(my_data.dim)

my_data.dim = map_col(my_data.dim)
mapp_data(my_data.dim)

#print(my_data.dim.keys())
      
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


Wall time: 1.02 s


## 6) Création des tables de fait

### Table de fait souhaits de formation

In [63]:
%%time

# Création des indicateurs de fomation prévisionnelle, ces derniers s'appuient sur les 4 tables liées aux souhaits de formation

# Les tables contenant l'ensemble des souhaits de formation 191119_TrainingCollectiveWishes et 191119_TrainingIndividualWishes vont être fusionnées dans la table wish_all
# Seules les clés externes et les attributs servant au calcule des indicateurs seront conservés
# La table 191119_TrainingIndividualWishes ne contenant pas l'indicateur "nbtrainees", nous considérons qu'une ligne équivaut à 1 nbtrainees

tcw_staging = tcw.copy()
tcw_staging = tcw_staging[["WishID","employeenumber", "FormationID", "nbtrainees", "nbmen", "nbwomen"]]
tiw_staging = tiw.copy()
tiw_staging = tiw_staging[["WishID", "ZY00.MATCLE","FormationID"]]
tiw_staging["nbtrainees"]=1

wish_all = tcw_staging.append(tiw_staging)

# Les tables contenant les demandes de formation associées à un plan vont être fusionnées
# Etape 1 récupérer les tables 191119_TrainingPlanCollectiveWishes et 191119_TrainingPlanIndividualWishes correspondant aux souhaits de formation associés à des plans
# Etape 2 on va supprimer les champs descriptifs: action, default_currency,hourly_wage_rage
# Etape 3 on fusionne les tables dans la table wish_plan

tpcw_staging = tpcw.copy()
tpcw_staging = tpcw_staging.drop(columns=["action", "default_currency","hourly_wage_rate"])
tpiw_staging = tpiw.copy()
tpiw_staging = tpiw_staging.drop(columns=["action", "default_currency"])

wish_plan = tpcw_staging.copy()
wish_plan = wish_plan.append(tpiw_staging)

# On fusionne les 2 tables créées afin de créer la table de fait wish_fact

wish_fact = pd.merge(wish_all, wish_plan, on = "WishID", how='left')
wish_fact.reset_index(drop=True, inplace=True)
my_data.import_fact("Indicateur Prévisonnel", wish_fact)

print(wish_fact)

#prev_fact = tpcw[["WishID", "nombre de stagiaire"]].copy()

#wish_fact = pd.merge(wish_fact, wish_plan[["WishID", "PlanID","wage_cost"]], how='left')

#wish_fact["wage_cost"]= wish_fact["wage_cost"].astype('float64')

#my_data.import_fact()
#dim_wish.loc[:, "Nb_demandes"]= 1

#my_dict_dataframe["wish_fact"] = wish_fact

      FormationID                                WishID ZY00.MATCLE  \
0          QHSSE9                               coll012         NaN   
1          ACHAC1                               coll009         NaN   
2         QHSRG21                               coll030         NaN   
3         QHSRG22                               coll006         NaN   
4         QHSRG22                               coll033         NaN   
...           ...                                   ...         ...   
21748      ACHAP1  db0ab039-4d31-481e-9fc7-5a991b67b61c     6902443   
21749     QHSRG21  3054ef36-1005-4efb-b1ca-b5d85ed116ca     6902476   
21750     QHSRG10  b01eeb7b-4a46-4e73-9fe7-dce8f74ad95f     6902476   
21751     QHSRG21                             REG20-108     6902476   
21752     QHSSE62                           PROJB20-367     6902520   

       employeenumber  nbmen  nbtrainees  nbwomen                   PlanID  \
0            107140.0    NaN          10      NaN        INVIVO GROUP

### Table de faits des formations

## Export des données vers le répertoire Transformed data

In [64]:
%%time
#for key, value in my_dict_dataframe.items():
 #       location = "Transformed data"
  #      file_name = str(key) + '.csv'
   #     location = os.path.join(location, file_name)  
    #    value.to_csv(location, encoding='utf16', index=False)
    
my_data.export()

Wall time: 978 ms
